http://publication.pravo.gov.ru/
---

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import wget
import time

file_out = 'base.txt'
START_PAGE = 'http://publication.pravo.gov.ru/SignatoryAuthority/court'
# START_PAGE = f'http://publication.pravo.gov.ru/SignatoryAuthority/region{17}' - ссылка для парсинга подзаконных актов регионов

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

params = {'method': 'GET',
          'credentials': 'include',
          'X-Requested-With': 'XMLHttpRequest'}

s = requests.Session()
s.params.update(params)
s.headers.update(headers)
s.proxies.update({})


def get_page(url):
    data = s.get(url).text
    soup = bs(data, 'lxml')
    return soup


def get_pagi():
    """Получаем количество страниц"""
    page = get_page(START_PAGE)
    data = page.find('div', {'class': 'page-nave-count'}).text
    item_count = int(data.split()[-1])
    return item_count


def get_items(data_soup):
    """извлекаем данные из страницы"""
    docs = []
    for items in data_soup.findAll('div', {'class': 'tr'}):
        data = items.findAll('a', {'class': 'choosedocument'})
        file = items.find('a', {'class': 'navigation'})
        if data:
            for i in data:
                url_doc = "http://publication.pravo.gov.ru{}".format(i.get('href'))
                title_doc = i.text
                file_url = "http://publication.pravo.gov.ru{}".format(file.get('href'))
                if title_doc:
                    doc = {"title": title_doc, "url": url_doc, "file": file_url}
                    docs.append(doc)
                    wget.download(file_url)
    return docs


with open(file_out, 'w', encoding='utf-8') as f:
    for page in range(1, get_pagi()//30):
        url = 'http://publication.pravo.gov.ru/Search/DocumentSearchResult?SearchObject.IsShowAppendPageCountList=true&' \
             'SearchObject.IsLastUpdateList=true&SearchObject.NavigationSignatoryAuthorityCode=court&' \
             'SearchObject.NavigationSignatoryAuthorityId=&SearchObject.NavigationSignatoryAuthorityCategory=&' \
             'SearchObject.SelectedSignatoryAuthorityId=00000000-0000-0000-0000-000000000000&SearchObject.RangeSize=30&' \
             'SearchObject.CurrentPageNumber={}&_=1597736552634'.format(page)
        soup = get_page(url)
        try:
            soup = get_page(url)
        except Exception as exc:
            print('FAIL: %r generated an exception: %s' % (url, exc))
            time.sleep(10)
        else:
            print('OK: %r page' % (url))
            
            
        for i in get_items(soup):
            print(i)
            f.write(str(i) + '\n')

OK: 'http://publication.pravo.gov.ru/Search/DocumentSearchResult?SearchObject.IsShowAppendPageCountList=true&SearchObject.IsLastUpdateList=true&SearchObject.NavigationSignatoryAuthorityCode=court&SearchObject.NavigationSignatoryAuthorityId=&SearchObject.NavigationSignatoryAuthorityCategory=&SearchObject.SelectedSignatoryAuthorityId=00000000-0000-0000-0000-000000000000&SearchObject.RangeSize=30&SearchObject.CurrentPageNumber=1&_=1597736552634' page
 12% [.........                                                                 ]  147456 / 1208340

KeyboardInterrupt: 

In [2]:
import pytesseract
from pdf2image import *
import glob
import os.path
import time

full_text = ""
counter = 0

pdfs = glob.glob(r"C:Pravo_gov_ru_pdfs//*.pdf")
pytesseract.pytesseract.tesseract_cmd = 'C://Program Files//Tesseract-OCR//tesseract.exe'

for pdf_path in pdfs:
    pages = convert_from_path(pdf_path, 
                              500, 
                              poppler_path=r'C://Users//icom1//OneDrive - icom//Рабочий стол//BERT Course//poppler-0.68.0//bin')

    for pageNum,imgBlob in enumerate(pages):
        text = pytesseract.image_to_string(imgBlob, lang='rus+eng')
        full_text += ' ' + text
    
    try:
        if not os.path.isfile(f'C://Pravo_gov_ru_txts//{counter}.txt'):
            with open(f'C://Pravo_gov_ru_txts//{counter}.txt', 'w') as the_file:
                the_file.write(full_text)
    except Exception as exc:
            time.sleep(10)
    else:
        print('OK: %r doc has been writed' % (counter))

    counter += 1

OK: 415 doc has been writed
